In [ ]:
#r "nuget: Microsoft.Extensions.Configuration.Json,7.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets,7.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console,7.0.0"
#r "nuget: Microsoft.Extensions.Logging.Debug,7.0.0"
#r "nuget: Microsoft.SemanticKernel,0.24.230918.1-preview"
#r "nuget: System.CommandLine,2.0.0-beta4.22272.1"
#r "nuget: YamlDotNet,13.5.2"

In [ ]:
using System;
using System.IO;
using System.Collections.Immutable;
using System.CommandLine;
using System.Text.RegularExpressions;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Events;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.SemanticFunctions;
using static Microsoft.SemanticKernel.SemanticFunctions.PromptTemplateConfig;

In [ ]:
var notebookDir = Environment.CurrentDirectory;
var parentDir = Directory.GetParent(notebookDir).FullName;

var builder = new ConfigurationBuilder()
    .SetBasePath(parentDir)  // Set the base path
    .AddJsonFile("config/appsettings.json", optional: false, reloadOnChange: true)  // Load the main appsettings.json file
    .AddJsonFile("appsettings.plugins.json", optional: true, reloadOnChange: true);  // Load the environment-specific appsettings file

var conf = builder.Build();

In [ ]:
ILoggerFactory loggerFactory = LoggerFactory.Create(builder =>
{
  builder.SetMinimumLevel(LogLevel.Warning)
          .AddConsole()
          .AddDebug();
});

In [ ]:
var serviceId = conf.GetValue<string>("serviceId");
var endpointType = conf.GetValue<string>("endpointType");
var serviceType = conf.GetValue<string>("serviceType");
var deploymentOrModelId = conf.GetValue<string>("deploymentOrModelId");
var apiKey = conf.GetValue<string>("apiKey");
var orgId = conf.GetValue<string>("orgId");


IKernel kernel = new KernelBuilder()
                     .WithLoggerFactory(loggerFactory)
                     .WithOpenAITextCompletionService(
                            modelId: deploymentOrModelId,
                            apiKey: apiKey,
                            orgId: orgId,
                            serviceId: serviceId)
                    .Build();

In [ ]:
var rootDirectory = conf.GetSection("SkillSettings:Root").Get<string>();
var pluginDirectories = conf.GetSection("SkillSettings:Plugins").Get<string[]>();

var pluginsRoot = Path.Combine(parentDir, rootDirectory!);
var pluginImport = kernel.ImportSemanticSkillFromDirectory(pluginsRoot, pluginDirectories!);

In [ ]:
string description = "create a kubernetes secret YAML for user/password";
var context = new ContextVariables();

string key = "input";
context.Set(key, description);

var result = await kernel.RunAsync(context, pluginImport["Kubernetes"]);
Console.WriteLine(result);